In [1]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import time

In [2]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QKDNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel
from sequence.protocols.qkd.BB84 import pair_bb84_protocols

In [3]:
class KeyManager():
    def __init__(self, timeline, keysize, num_keys):
        self.timeline = timeline
        self.lower_protocols = []
        self.keysize = keysize
        self.num_keys = num_keys
        self.keys = []
        self.times = []
        
    def send_request(self):
        for p in self.lower_protocols:
            p.push(self.keysize, self.num_keys)
            
    def pop(self, msg):
        self.keys.append(msg)
        self.times.append(self.timeline.now())

In [4]:
def test(sim_time, keysize):
    """
    sim_time: duration of simulation time (ms)
    """
    tl = Timeline(sim_time * 1e9)
    
    n1 = QKDNode("n1", tl)
    n2 = QKDNode("n2", tl)
    pair_bb84_protocols(n1.sifting_protocol, n2.sifting_protocol)
    
    cc = ClassicalChannel("cc_n1_n2", tl, 2e-4, 1e3)
    cc.set_ends(n1, n2)
    qc = QuantumChannel("qc_n1_n2", tl, 1e-5, 1e3)
    qc.set_ends(n1, n2)
    
    km1 = KeyManager(tl, keysize, 10)
    km1.lower_protocols.append(n1.sifting_protocol)
    n1.sifting_protocol.upper_protocols.append(km1)
    km2 = KeyManager(tl, keysize, 10)
    km2.lower_protocols.append(n2.sifting_protocol)
    n2.sifting_protocol.upper_protocols.append(km2)
    
    tl.init()
    km1.send_request()
    
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    plt.plot(km1.times, range(1, len(km1.keys) + 1), marker="o")
    plt.show()
    print("key error rates:")
    for i, e in enumerate(n1.sifting_protocol.error_rates):
        print("\tkey {}:\t{}%".format(i + 1, e * 100))

In [5]:
interactive_plot = interact(test, sim_time=(100, 1000, 100), keysize=[128, 256, 512])
interactive_plot

interactive(children=(IntSlider(value=500, description='sim_time', max=1000, min=100, step=100), Dropdown(desc…

<function __main__.test(sim_time, keysize)>